In [32]:
%matplotlib inline
import matplotlib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import glob
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedShuffleSplit
from skimage.io import imread
from skimage.transform import resize
from tensorflow.python.framework.ops import reset_default_graph
import os
import subprocess
import itertools
from datetime import datetime

def onehot(t, num_classes):
    out = np.zeros((t.shape[0], num_classes))
    for row, col in enumerate(t):
        out[row, col] = 1
    return out

In [33]:
class load_data():
    # data_train, data_test and le are public
    def __init__(self, train_path, test_path, image_paths, image_shape=(128, 128)):
        train_df = pd.read_csv(train_path)
        test_df = pd.read_csv(test_path)
        image_paths = image_paths
        image_shape = image_shape
        self._load(train_df, test_df, image_paths, image_shape)
        
    def _load(self, train_df, test_df, image_paths, image_shape):
        print("loading data ...")
        # load train.csv
        path_dict = self._path_to_dict(image_paths) # numerate image paths and make it a dict
        # merge image paths with data frame
        train_image_df = self._merge_image_df(train_df, path_dict)
        test_image_df = self._merge_image_df(test_df, path_dict)
        # label encoder-decoder (self. because we need it later)
        self.le = LabelEncoder().fit(train_image_df['species'])
        # labels for train
        t_train = self.le.transform(train_image_df['species'])
        # getting data
        train_data = self._make_dataset(train_image_df, image_shape, t_train)
        test_data = self._make_dataset(test_image_df, image_shape)        
        # need to reformat the train for validation split reasons in the batch_generator
        self.train = self._format_dataset(train_data, for_train=True)
        self.test = self._format_dataset(test_data, for_train=False)
        print("data loaded") 
        

    def _path_to_dict(self, image_paths):
        path_dict = dict()
        for image_path in image_paths:
            num_path = int(os.path.basename(image_path[:-4]))
            path_dict[num_path] = image_path
        return path_dict

    def _merge_image_df(self, df, path_dict):
        split_path_dict = dict()
        for index, row in df.iterrows():
            split_path_dict[row['id']] = path_dict[row['id']]
        image_frame = pd.DataFrame(list(split_path_dict.values()), columns=['image'])
        df_image =  pd.concat([image_frame, df], axis=1)
        return df_image
    

    def _make_dataset(self, df, image_shape, t_train=None):
        if t_train is not None:
            print("loading train ...")
        else:
            print("loading test ...")
        # make dataset
        data = dict()
        # merge image with 3x64 features
        for i, dat in enumerate(df.iterrows()):
            index, row = dat
            sample = dict()
            if t_train is not None:
                features = row.drop(['id', 'species', 'image'], axis=0).values
            else:
                features = row.drop(['id', 'image'], axis=0).values
            sample['margin'] = features[:64]
            sample['shape'] = features[64:128]
            sample['texture'] = features[128:]
            if t_train is not None:
                sample['t'] = np.asarray(t_train[i], dtype='int32')
            image = imread(row['image'], as_grey=True)
            image = resize(image, output_shape=image_shape)
            image = np.expand_dims(image, axis=2)
            sample['image'] = image   
            data[row['id']] = sample
            if i % 100 == 0:
                print("\t%d of %d" % (i, len(df)))
        return data

    def _format_dataset(self, df, for_train):
        # making arrays with all data in, is nessesary when doing validation split
        data = dict()
        value = list(df.values())[0]
        img_tot_shp = tuple([len(df)] + list(value['image'].shape))
        data['images'] = np.zeros(img_tot_shp, dtype='float32')
        feature_tot_shp = (len(df), 64)
        data['margins'] = np.zeros(feature_tot_shp, dtype='float32')
        data['shapes'] = np.zeros(feature_tot_shp, dtype='float32')
        data['textures'] = np.zeros(feature_tot_shp, dtype='float32')
        if for_train:
            data['ts'] = np.zeros((len(df),), dtype='int32')
        else:
            data['ids'] = np.zeros((len(df),), dtype='int32')
        for i, pair in enumerate(df.items()):
            key, value = pair
            data['images'][i] = value['image']
            data['margins'][i] = value['margin']
            data['shapes'][i] = value['shape']
            data['textures'][i] = value['texture']
            if for_train:
                data['ts'][i] = value['t']
            else:
                data['ids'][i] = key
        return data

In [34]:

BASE_PATH = "/Users/avickbiswas/DataScience/Kaggle/leaves\ classification/"
TRAIN_PATH = "train.csv"
TEST_PATH = "test.csv"
IMAGE_PATHS = glob.glob("images/*.jpg")
NUM_CLASSES = 99
IMAGE_SHAPE = (128, 128, 1)
NUM_FEATURES = 64 # for all three features, margin, shape and texture
# train holds both X (input) and t (target/truth)
data = load_data(train_path=TRAIN_PATH, test_path=TEST_PATH,
                 image_paths=IMAGE_PATHS, image_shape=IMAGE_SHAPE[:2])
# to visualize the size of the dimensions of the data
print
print("@@@Shape checking of data sets@@@")
print
print("TRAIN")
print("\timages\t%s%f" % (data.train['images'].shape, data.train['images'].mean())) 
print("\tmargins\t%s\t%f" % (data.train['margins'].shape, data.train['margins'].mean()))
print("\tshapes\t%s\t%f" % (data.train['shapes'].shape, data.train['shapes'].mean()))
print("\ttextures%s\t%f" % (data.train['textures'].shape, data.train['textures'].mean()))
print("\tts\t %s" % (data.train['ts'].shape))
print("\twhile training, batch_generator will onehot encode ts to (batch_size, num_classes)")
print
print("TEST")
print("\timages\t%s\t%f" % (data.test['images'].shape, data.test['images'].mean()))
print("\tmargins\t%s\t%f" % (data.test['margins'].shape, data.test['margins'].mean()))
print("\tshapes\t%s\t%f" % (data.test['shapes'].shape, data.test['shapes'].mean()))
print("\ttextures%s\t%f" % (data.test['textures'].shape, data.test['textures'].mean()))
print("\tids\t%s" % (data.test['ids'].shape))

loading data ...
loading train ...
	0 of 990
	100 of 990
	200 of 990
	300 of 990
	400 of 990
	500 of 990
	600 of 990
	700 of 990
	800 of 990
	900 of 990
loading test ...
	0 of 594
	100 of 594
	200 of 594
	300 of 594
	400 of 594
	500 of 594
data loaded
@@@Shape checking of data sets@@@
TRAIN
	images	(990, 128, 128, 1)0.462174
	margins	(990, 64)	0.015625
	shapes	(990, 64)	0.000607
	textures(990, 64)	0.015625
	ts	 990
	while training, batch_generator will onehot encode ts to (batch_size, num_classes)
TEST
	images	(594, 128, 128, 1)	0.463148
	margins	(594, 64)	0.015625
	shapes	(594, 64)	0.000604
	textures(594, 64)	0.015625
	ids	594


In [46]:
from sklearn.cross_validation import StratifiedShuffleSplit as SSS

class batch_generator():
    def __init__(self, data, batch_size=64, num_classes=99,
                 num_iterations=5e3, num_features=64, seed=42, val_size=0.1):
        print("initiating batch generator")
        self._train = data.train
        self._test = data.test
        # get image size
        value = self._train['images'][0]
        self._image_shape = list(value.shape)
        self._batch_size = batch_size
        self._num_classes = num_classes
        self._num_iterations = num_iterations
        self._num_features = num_features
        self._seed = seed
        self._val_size = 0.1
        self._valid_split()
        print("batch generator initiated ...")

    def _valid_split(self):
        self._idcs_train, self._idcs_valid = next(iter(
            SSS(self._train['ts'], n_iter=1, test_size=self._val_size, random_state=self._seed)))
        
    def _shuffle_train(self):
        np.random.shuffle(self._idcs_train)

    def _batch_init(self, purpose):
        assert purpose in ['train', 'valid', 'test']
        batch_holder = dict()
        batch_holder['margins'] = np.zeros((self._batch_size, self._num_features), dtype='float32')
        batch_holder['shapes'] = np.zeros((self._batch_size, self._num_features), dtype='float32')
        batch_holder['textures'] = np.zeros((self._batch_size, self._num_features), dtype='float32')
        batch_holder['images'] = np.zeros(tuple([self._batch_size] + self._image_shape), dtype='float32')
        if (purpose == "train") or (purpose == "valid"):
            batch_holder['ts'] = np.zeros((self._batch_size, self._num_classes), dtype='float32')          
        else:
            batch_holder['ids'] = []
        return batch_holder

    def gen_valid(self):
        batch = self._batch_init(purpose='train')
        i = 0
        for idx in self._idcs_valid:
            batch['margins'][i] = self._train['margins'][idx]
            batch['shapes'][i] = self._train['shapes'][idx]
            batch['textures'][i] = self._train['textures'][idx]
            batch['images'][i] = self._train['images'][idx]
            batch['ts'][i] = onehot(np.asarray([self._train['ts'][idx]], dtype='float32'), self._num_classes)
            i += 1
            if i >= self._batch_size:
                yield batch, i
                batch = self._batch_init(purpose='valid')
                i = 0
        if i != 0:
            yield batch, i

    def gen_test(self):
        batch = self._batch_init(purpose='test')
        i = 0
        for idx in range(len(self._test['ids'])):
            batch['margins'][i] = self._test['margins'][idx]
            batch['shapes'][i] = self._test['shapes'][idx]
            batch['textures'][i] = self._test['textures'][idx]
            batch['images'][i] = self._test['images'][idx]
            batch['ids'].append(self._test['ids'][idx])
            i += 1
            if i >= self._batch_size:
                yield batch, i
                batch = self._batch_init(purpose='test')
                i = 0
        if i != 0:
            yield batch, i
            

    def gen_train(self):
        batch = self._batch_init(purpose='train')
        iteration = 0
        i = 0
        while True:
            # shuffling all batches
            self._shuffle_train()
            for idx in self._idcs_train:
                # extract data from dict
                batch['margins'][i] = self._train['margins'][idx]
                batch['shapes'][i] = self._train['shapes'][idx]
                batch['textures'][i] = self._train['textures'][idx]
                batch['images'][i] = self._train['images'][idx]
                batch['ts'][i] = onehot(np.asarray([self._train['ts'][idx]], dtype='float32'), self._num_classes)
                i += 1
                if i >= self._batch_size:
                    yield batch
                    batch = self._batch_init(purpose='train')
                    i = 0
                    iteration += 1
                    if iteration >= self._num_iterations:
                        break

In [49]:
dummy_batch_gen = batch_generator(data, batch_size=64, num_classes=99, num_iterations=5e3, seed=42)
train_batch = next(dummy_batch_gen.gen_train())
valid_batch, i = next(dummy_batch_gen.gen_valid())
test_batch, i = next(dummy_batch_gen.gen_test())

print
print("@@@Shape/mean checking of batches@@@")
print
print("TRAIN")
print("\timages,", train_batch['images'].shape)
print("\tmargins,", train_batch['margins'].shape)
print("\tshapes,", train_batch['shapes'].shape)
print("\ttextures,", train_batch['textures'].shape)
print("\tts,", train_batch['ts'].shape)
print
print("VALID")
print("\timages,", valid_batch['images'].shape)
print("\tmargins,", valid_batch['margins'].shape)
print("\tshapes,", valid_batch['shapes'].shape)
print("\ttextures,", valid_batch['textures'].shape)
print("\tts,", valid_batch['ts'].shape)
print
print("TEST")
print("\timages,", test_batch['images'].shape)
print("\tmargins,", test_batch['margins'].shape)
print("\tshapes,", test_batch['shapes'].shape)
print("\ttextures,", test_batch['textures'].shape)
print("\tids,", len(test_batch['ids']))
# notice that mean is very different, which is why we use batch_norm in all input data in model

initiating batch generator
batch generator initiated ...
@@@Shape/mean checking of batches@@@
TRAIN
	images, (64, 128, 128, 1)
	margins, (64, 64)
	shapes, (64, 64)
	textures, (64, 64)
	ts, (64, 99)
VALID
	images, (64, 128, 128, 1)
	margins, (64, 64)
	shapes, (64, 64)
	textures, (64, 64)
	ts, (64, 99)
TEST
	images, (64, 128, 128, 1)
	margins, (64, 64)
	shapes, (64, 64)
	textures, (64, 64)
	ids, 64


/Users/avickbiswas/anaconda/envs/tensorflow/lib/python3.5/site-packages/ipykernel/__main__.py:21: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


In [50]:
# contrib layers similar to wrappings used in Lasagne (for theano) or Keras
from tensorflow.contrib.layers import fully_connected, convolution2d, flatten, batch_norm, max_pool2d, dropout
from tensorflow.python.ops.nn import relu, elu, relu6, sigmoid, tanh, softmax
from tensorflow.python.ops.nn import dynamic_rnn

In [51]:
# wrapping conv with batch_norm
def conv(l_in, num_outputs, kernel_size, scope, stride=1):
    return convolution2d(l_in, num_outputs=num_outputs, kernel_size=kernel_size,
                         stride=stride, normalizer_fn=batch_norm, scope=scope)

# pre-activation: http://arxiv.org/abs/1603.05027
# wrapping convolutions and batch_norm
def conv_pre(l_in, num_outputs, kernel_size, scope, stride=1):
    l_norm = batch_norm(l_in)
    l_relu = relu(l_norm)
    return convolution2d(l_relu, num_outputs=num_outputs, kernel_size=kernel_size,
                         stride=stride, activation_fn=None, scope=scope)
# easy to use pool function
def pool(l_in, scope, kernel_size=(3, 3)):
    return max_pool2d(l_in, kernel_size=kernel_size, scope=scope) # (3, 3) has shown to work better than (2, 2)

In [52]:
# hyperameters of the model
height, width, channels = IMAGE_SHAPE
# resetting the graph ...
reset_default_graph()

# Setting up placeholder, this is where your data enters the graph!
x_image_pl = tf.placeholder(tf.float32, [None, height, width, channels], name="x_image_pl")
x_margin_pl = tf.placeholder(tf.float32, [None, NUM_FEATURES], name="x_margin_pl")
x_shape_pl = tf.placeholder(tf.float32, [None, NUM_FEATURES], name="x_shape_pl")
x_texture_pl = tf.placeholder(tf.float32, [None, NUM_FEATURES], name="x_texture_pl")
is_training_pl = tf.placeholder(tf.bool, name="is_training_pl")

# Building the layers of the neural network
# we define the variable scope, so we more easily can recognise our variables later

## IMAGE
#l_conv1_a = conv(x_image_pl, 16, (5, 5), scope="l_conv1_a")
#l_pool1 = pool(l_conv1_a, scope="l_pool1")
#l_conv2_a = conv(l_pool1, 16, (5, 5), scope="l_conv2_a")
#l_pool2 = pool(l_conv2_a, scope="l_pool2")
#l_conv3_a = conv(l_pool2, 16, (5, 5), scope="l_conv3_a")
#l_pool3 = pool(l_conv3_a, scope="l_pool3")
#l_conv4_a = conv(l_pool3, 16, (5, 5), scope="l_conv4_a")
#l_pool4 = pool(l_conv3_a, scope="l_pool4")
#l_flatten = flatten(l_pool4, scope="flatten")

## RNN
# define the cell of your RNN
#shape_cell = tf.nn.rnn_cell.GRUCell(100)
# run the RNN as outputs, state = tf.nn.dynamic_rnn(cell, ...)
# given we run many-to-one we only care about the last state, so only
# shape_state is defined
#_, shape_state = tf.nn.dynamic_rnn(cell=shape_cell,
#    inputs=tf.expand_dims(batch_norm(x_shape_pl), 2), dtype=tf.float32, scope="shape_rnn")

## COMBINE
# use margin, shape and texture only
features = tf.concat(concat_dim=1, values=[x_margin_pl, x_shape_pl, x_texture_pl], name="features")
# uncomment to use image only
#features = l_flatten
# uncomment to use margin, rnn_state on shape and texture only
#features = tf.concat(concat_dim=1, values=[x_margin_pl, shape_state, x_texture_pl], name="features")
features = batch_norm(features, scope='features_bn')
#l2 = fully_connected(features, num_outputs=256, activation_fn=relu,
#                     normalizer_fn=batch_norm, scope="l2")
#l2 = dropout(l2, is_training=is_training_pl, scope="l2_dropout")
y = fully_connected(features, NUM_CLASSES, activation_fn=softmax, scope="y")

# add TensorBoard summaries for all variables
tf.contrib.layers.summarize_variables()

[<tf.Tensor 'features_bn/beta_summary:0' shape=() dtype=string>,
 <tf.Tensor 'features_bn/moving_mean_summary:0' shape=() dtype=string>,
 <tf.Tensor 'features_bn/moving_variance_summary:0' shape=() dtype=string>,
 <tf.Tensor 'y/weights_summary:0' shape=() dtype=string>,
 <tf.Tensor 'y/biases_summary:0' shape=() dtype=string>]

In [55]:
# PRINT NETWORK (good practice to also include outcommented code when using it)

print("x_image_pl,", x_image_pl.get_shape())
print("x_margin_pl,", x_margin_pl.get_shape())
print("x_shape_pl,", x_shape_pl.get_shape())
print("x_texture_pl,", x_texture_pl.get_shape())
print("features,", features.get_shape())
print("y,", y.get_shape())

# for the MLP
#print "l2,", l2.get_shape()
# for the RNN
#print "shape_state,", shape_state.get_shape()
# for the CNN
#print "l_conv1_a,", l_conv1_a.get_shape()
#...
#print "l_pool4,", l_pool4.get_shape()
#print "l_flatten,", l_flatten.get_shape()

x_image_pl, (?, 128, 128, 1)
x_margin_pl, (?, 64)
x_shape_pl, (?, 64)
x_texture_pl, (?, 64)
features, (?, 192)
y, (?, 99)


In [56]:
clip_norm = 1
# y_ is a placeholder variable taking on the value of the target batch.
ts_pl = tf.placeholder(tf.float32, [None, NUM_CLASSES], name="targets_pl")
lr_pl = tf.placeholder(tf.float32, [], name="learning_rate_pl")

def loss_and_acc(preds):
    # computing cross entropy per sample
    cross_entropy = -tf.reduce_sum(ts_pl * tf.log(preds+1e-10), reduction_indices=[1])
    # averaging over samples
    loss = tf.reduce_mean(cross_entropy)
    # if you want regularization
    #reg_scale = 0.0001
    #regularize = tf.contrib.layers.l2_regularizer(reg_scale)
    #params = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES)
    #reg_term = sum([regularize(param) for param in params])
    #loss += reg_term
    # calculate accuracy
    argmax_y = tf.to_int32(tf.argmax(preds, dimension=1))
    argmax_t = tf.to_int32(tf.argmax(ts_pl, dimension=1))
    correct = tf.to_float(tf.equal(argmax_y, argmax_t))
    accuracy = tf.reduce_mean(correct)
    return loss, accuracy, argmax_y

# loss, accuracy and prediction
loss, accuracy, prediction = loss_and_acc(y)

loss_valid = loss
accuracy_valid = accuracy
loss_valid, accuracy_valid, _ = loss_and_acc(y)

# defining our optimizer
optimizer = tf.train.AdamOptimizer(learning_rate=0.001)

# applying the gradients
grads_and_vars = optimizer.compute_gradients(loss)
gradients, variables = zip(*grads_and_vars)  # unzip list of tuples
clipped_gradients, global_norm = (
    tf.clip_by_global_norm(gradients, clip_norm) )
clipped_grads_and_vars = zip(clipped_gradients, variables)

# make training op for applying the gradients
train_op = optimizer.apply_gradients(clipped_grads_and_vars)

# make tensorboard summeries
tf.scalar_summary('train/global gradient norm', global_norm)
tf.scalar_summary('train/loss', loss)
tf.scalar_summary('train/accuracy', accuracy)
tf.scalar_summary('validation/loss', loss_valid)
tf.scalar_summary('validation/accuracy', accuracy_valid)


Instructions for updating:
Please switch to tf.summary.scalar. Note that tf.summary.scalar uses the node name instead of the tag. This means that TensorFlow will automatically de-duplicate summary names based on the scope they are created in. Also, passing a tensor or list of tags to a scalar summary op is no longer supported.
Instructions for updating:
Please switch to tf.summary.scalar. Note that tf.summary.scalar uses the node name instead of the tag. This means that TensorFlow will automatically de-duplicate summary names based on the scope they are created in. Also, passing a tensor or list of tags to a scalar summary op is no longer supported.
Instructions for updating:
Please switch to tf.summary.scalar. Note that tf.summary.scalar uses the node name instead of the tag. This means that TensorFlow will automatically de-duplicate summary names based on the scope they are created in. Also, passing a tensor or list of tags to a scalar summary op is no longer supported.
Instructions 

<tf.Tensor 'ScalarSummary_4:0' shape=() dtype=string>

In [58]:
#Test the forward pass
_img_shape = tuple([45]+list(IMAGE_SHAPE))
_feature_shape = (45, NUM_FEATURES)
_x_image = np.random.normal(0, 1, _img_shape).astype('float32') #dummy data
_x_margin = np.random.normal(0, 1, _feature_shape).astype('float32')
_x_shape = np.random.normal(0, 1, _feature_shape).astype('float32')
_x_texture = np.random.normal(0, 1, _feature_shape).astype('float32')

# restricting memory usage, TensorFlow is greedy and will use all memory otherwise
gpu_opts = tf.GPUOptions(per_process_gpu_memory_fraction=0.2)
# initialize the Session
sess = tf.Session(config=tf.ConfigProto(gpu_options=gpu_opts))
# test the forward pass
sess.run(tf.global_variables_initializer())
feed_dict = {x_image_pl: _x_image,
             x_margin_pl: _x_margin,
             x_shape_pl: _x_shape,
             x_texture_pl: _x_texture,
             is_training_pl: False}
res_forward_pass = sess.run(fetches=[y], feed_dict=feed_dict)
print("y", res_forward_pass[0].shape)

y (45, 99)


In [62]:
#Training Loop
BATCH_SIZE = 64
ITERATIONS = 1e4
LOG_FREQ = 10
VALIDATION_SIZE = 0.1 # 0.1 is ~ 100 samples for valition
SEED = 42
DROPOUT = False
LEARNING_RATE = 0.0005
VALID_EVERY = 100

batch_gen = batch_generator(data, batch_size=BATCH_SIZE, num_classes=NUM_CLASSES,
                            num_iterations=ITERATIONS, seed=SEED, val_size=VALIDATION_SIZE)

# setup and write summaries
timestamp = datetime.now().strftime("%Y%m%d-%H%M%S")
summaries_path = "tensorboard/%s/logs" % (timestamp)
summaries = tf.summary.merge_all();#tf.merge_all_summaries()
summarywriter = tf.train.SummaryWriter(summaries_path, sess.graph)

train_loss = []
train_acc = []
print("\ttrain_loss \ttrain_acc \tvalid_loss \tvalid_acc")
for i, batch_train in enumerate(batch_gen.gen_train()):
    if i>=ITERATIONS:
        break
    fetches_train = [train_op, loss, accuracy, summaries]
    feed_dict_train = {
        x_image_pl: batch_train['images'],
        x_margin_pl: batch_train['margins'],
        x_shape_pl: batch_train['shapes'],
        x_texture_pl: batch_train['textures'],
        ts_pl: batch_train['ts'],
        is_training_pl: DROPOUT,
        lr_pl: LEARNING_RATE,
        
    }
    res_train = sess.run(fetches=fetches_train, feed_dict=feed_dict_train)
    if i % LOG_FREQ == 0:
        summarywriter.add_summary(res_train[3], i)
    train_loss.append(res_train[1])
    train_acc.append(res_train[2])
    
    # validate
    if i % VALID_EVERY == 0:
        cur_acc = 0
        cur_loss = 0
        tot_num = 0
        # batch validation
        for batch_valid, num in batch_gen.gen_valid():
            # fetches and feed_dict for validation
            fetches_valid = [loss_valid, accuracy_valid, summaries]
            feed_dict_valid = {
                x_image_pl: batch_valid['images'],
                x_margin_pl: batch_valid['margins'],
                x_shape_pl: batch_valid['shapes'],
                x_texture_pl: batch_valid['textures'],
                ts_pl: batch_valid['ts'],
                is_training_pl: False,
            }
            # run validation
            res_valid = sess.run(fetches=fetches_valid, feed_dict=feed_dict_valid)
            # tensorboard and costs
            summarywriter.add_summary(res_valid[2], i)
            cur_loss += res_valid[0]*num
            cur_acc += res_valid[1]*num
            tot_num += num
        valid_loss = cur_loss / float(tot_num)
        valid_acc = (cur_acc / float(tot_num)) * 100
        train_loss = sum(train_loss) / float(len(train_loss))
        train_acc = sum(train_acc) / float(len(train_acc)) * 100
        print("%d:\t  %.2f\t\t  %.1f\t\t  %.2f\t\t  %.1f" % (i, train_loss, train_acc, valid_loss, valid_acc))
        train_loss = []
        train_acc = []

initiating batch generator
batch generator initiated ...
Instructions for updating:
Please switch to tf.summary.FileWriter. The interface and behavior is the same; this is just a rename.
	train_loss 	train_acc 	valid_loss 	valid_acc
0:	  0.00		  100.0		  0.01		  84.0


/Users/avickbiswas/anaconda/envs/tensorflow/lib/python3.5/site-packages/ipykernel/__main__.py:21: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


100:	  0.00		  100.0		  0.01		  84.0
200:	  0.00		  100.0		  0.01		  84.0
300:	  0.00		  100.0		  0.01		  84.0
400:	  0.00		  100.0		  0.01		  84.0
500:	  0.00		  100.0		  0.01		  84.0
600:	  0.00		  100.0		  0.01		  84.0
700:	  0.00		  100.0		  0.01		  84.0
800:	  0.00		  100.0		  0.01		  84.0
900:	  0.00		  100.0		  0.01		  84.0
1000:	  0.00		  100.0		  0.01		  84.0
1100:	  0.00		  100.0		  0.01		  84.0
1200:	  0.00		  100.0		  0.01		  84.0
1300:	  0.00		  100.0		  0.01		  84.0
1400:	  0.00		  100.0		  0.01		  84.0
1500:	  0.00		  100.0		  0.01		  84.0
1600:	  0.00		  100.0		  0.01		  84.0
1700:	  0.00		  100.0		  0.01		  84.0
1800:	  0.00		  100.0		  0.01		  84.0
1900:	  0.00		  100.0		  0.01		  84.0
2000:	  0.00		  100.0		  0.01		  84.0
2100:	  0.00		  100.0		  0.01		  84.0
2200:	  0.00		  100.0		  0.01		  84.0
2300:	  0.00		  100.0		  0.01		  84.0
2400:	  0.00		  100.0		  0.01		  84.0
2500:	  0.00		  100.0		  0.01		  84.0
2600:	  0.00		  100.0		  0.01		  84.0
2700:	  0.00		  100.0

In [63]:
# GET PREDICTIONS
# containers to collect ids and predictions
ids_test = []
preds_test = []
# run like with validation
for batch_test, num in batch_gen.gen_test():
    # fetching for test we only need y
    fetches_test = [y]
    # same as validation, but no batch['ts']
    feed_dict_test = {
        x_image_pl: batch_test['images'],
        x_margin_pl: batch_test['margins'],
        x_shape_pl: batch_test['shapes'],
        x_texture_pl: batch_test['textures'],
        is_training_pl: False
    }
    # get the result
    res_test = sess.run(fetches=fetches_test, feed_dict=feed_dict_test)
    y_out = res_test[0]
    ids_test.append(batch_test['ids'])
    if num!=len(y_out):
        # in case of the last batch, num will be less than batch_size
        y_out = y_out[:num]
    preds_test.append(y_out)
# concatenate it all, to form one list/array
ids_test = list(itertools.chain.from_iterable(ids_test))
preds_test = np.concatenate(preds_test, axis=0)
assert len(ids_test) == len(preds_test)

In [64]:
preds_df = pd.DataFrame(preds_test, columns=data.le.classes_)
ids_test_df = pd.DataFrame(ids_test, columns=["id"])
submission = pd.concat([ids_test_df, preds_df], axis=1)
submission.to_csv('submission_mlp.csv', index=False)
# below prints the submission, can be removed and replaced with code block below
submission

,id,Acer_Capillipes,Acer_Circinatum,Acer_Mono,Acer_Opalus,Acer_Palmatum,Acer_Pictum,Acer_Platanoids,Acer_Rubrum,Acer_Rufinerve,...,Salix_Fragilis,Salix_Intergra,Sorbus_Aria,Tilia_Oliveri,Tilia_Platyphyllos,Tilia_Tomentosa,Ulmus_Bergmanniana,Viburnum_Tinus,Viburnum_x_Rhytidophylloides,Zelkova_Serrata
0,1026,2.869020e-13,1.529277e-10,3.497055e-09,3.752200e-09,8.665882e-09,1.668734e-15,1.253632e-11,1.692365e-10,5.721615e-14,...,4.268308e-14,9.879059e-07,5.018530e-12,3.757866e-09,6.097649e-06,1.296256e-09,3.317226e-14,6.300510e-06,4.647927e-09,5.881330e-06
1,4,1.159570e-12,8.310549e-13,3.229270e-15,4.123330e-07,4.229517e-13,1.825746e-16,1.171082e-11,1.719328e-13,2.621086e-14,...,4.097174e-14,1.478120e-08,2.576582e-12,1.178369e-13,6.354355e-10,4.283691e-14,1.008377e-15,1.131792e-14,2.047118e-13,6.398953e-14
2,1029,5.695827e-08,8.574796e-09,6.509347e-11,1.547787e-11,1.921026e-07,2.038853e-10,1.342976e-05,1.524260e-10,1.485938e-05,...,8.604357e-02,1.091695e-08,1.416735e-04,5.031326e-08,2.656954e-05,4.285481e-06,9.209554e-03,1.420646e-08,1.433977e-08,2.518181e-09
3,7,3.855745e-13,6.281097e-12,1.426525e-11,1.040503e-09,1.433467e-10,2.495290e-12,8.599153e-09,1.126502e-11,9.465763e-13,...,1.065881e-13,1.152906e-12,6.107722e-13,7.608827e-13,2.042700e-13,6.974107e-09,2.163322e-13,1.269684e-09,3.280878e-13,1.685711e-08
4,9,6.289131e-11,9.998888e-01,9.492183e-11,4.485563e-10,6.429574e-05,5.592501e-09,2.558854e-11,7.623985e-07,1.593226e-07,...,3.396650e-10,6.050024e-10,1.039362e-10,1.682971e-11,9.298544e-15,1.238260e-10,2.479792e-10,1.954589e-13,9.756926e-13,1.767668e-05
5,1035,1.818157e-11,6.863546e-09,2.173861e-09,4.438699e-08,2.217040e-10,5.321795e-09,8.048378e-13,7.392057e-14,1.955893e-14,...,4.699236e-13,6.840367e-11,4.944857e-11,2.903605e-12,1.078269e-11,1.067115e-10,5.698240e-11,1.130901e-09,1.218257e-08,1.763703e-09
6,12,1.134462e-14,5.331436e-07,4.497413e-11,1.467479e-13,4.536607e-10,8.801194e-14,1.253105e-07,8.469607e-12,9.058447e-07,...,4.105444e-11,1.528023e-12,1.819910e-09,1.115343e-11,4.180719e-11,1.226446e-08,1.039584e-04,1.779851e-10,4.638010e-10,1.362704e-05
7,13,4.846606e-12,2.529955e-11,1.497237e-14,2.621346e-14,1.929146e-11,4.638825e-16,5.184325e-10,8.176667e-13,6.479242e-09,...,1.929581e-10,2.864881e-13,3.057113e-08,2.215729e-11,7.997122e-08,2.561640e-09,2.334679e-07,8.070605e-10,4.601576e-12,1.653976e-13
8,1038,2.175138e-08,5.019227e-11,2.582438e-07,3.543059e-05,8.564473e-13,7.389378e-07,6.726415e-08,1.746448e-13,1.388468e-13,...,4.369506e-11,2.025135e-13,1.079750e-10,1.002365e-11,6.731412e-14,2.736050e-09,1.360797e-13,2.705563e-15,3.459092e-10,4.684759e-10
9,16,1.413082e-08,7.833825e-10,8.155917e-10,9.796035e-01,4.727880e-09,2.875598e-11,3.462835e-08,3.312818e-06,9.231528e-09,...,1.673009e-06,1.388219e-07,7.266534e-08,1.893304e-07,4.795458e-06,5.396638e-06,8.016678e-10,1.999327e-09,5.937847e-06,6.150653e-08
